In [ ]:
import ee
import geemap
import numpy as np


ee.Authenticate()
ee.Initialize(project='socd-liuziyan')


dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V4_1')
elevation_collection = dataset.select('DSM')


proj = elevation_collection.first().select(0).projection()
elevation = elevation_collection.mosaic().setDefaultProjection(proj)

slope = ee.Terrain.slope(elevation)


kernel = ee.Kernel.circle(radius=7000, units='meters')


elevation_max = elevation.reduceNeighborhood(
    reducer=ee.Reducer.max(),
    kernel=kernel
)
elevation_min = elevation.reduceNeighborhood(
    reducer=ee.Reducer.min(),
    kernel=kernel
)
local_elevation_range = elevation_max.subtract(elevation_min)


lat = ee.Image.pixelLonLat().select('latitude')
lat_mask = lat.gt(23).Or(lat.lt(-19))


def classify_mountains():

    mountain_type = ee.Image.constant(0)
    
    type1 = elevation.gt(4500)
    mountain_type = mountain_type.where(type1, 1)
    
    type2 = elevation.gte(3500).And(elevation.lte(4500))
    mountain_type = mountain_type.where(type2, 2)
    
    type3 = elevation.gte(2500).And(elevation.lte(3500))
    mountain_type = mountain_type.where(type3, 3)
    
    type4 = elevation.gte(1500).And(elevation.lte(2500)).And(slope.gte(2))
    mountain_type = mountain_type.where(type4, 4)
    
    type5_elevation = elevation.gte(1000).And(elevation.lte(1500))
    type5_slope = slope.gte(5)
    type5_relief = local_elevation_range.gt(300)
    type5 = type5_elevation.And(type5_slope.Or(type5_relief))
    mountain_type = mountain_type.where(type5, 5)
    
    type6_elevation = elevation.gte(300).And(elevation.lte(1000))
    type6_relief = local_elevation_range.gt(300)
    type6 = type6_elevation.And(type6_relief).And(lat_mask)
    mountain_type = mountain_type.where(type6, 6)
    
    return mountain_type

mountain_classification = classify_mountains()

mountain_mask = mountain_classification.gt(0)
mountain_areas = mountain_classification.updateMask(mountain_mask)

elevation_vis = {
    'min': 0,
    'max': 5000,
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

mountain_vis = {
    'min': 1,
    'max': 6,
    'palette': ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#66bd63']
}

Map = geemap.Map(center=[0, 0], zoom=2)

Map.addLayer(elevation, elevation_vis, 'ALOS AW3D30 Elevation')
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['blue', 'yellow', 'red']}, 'Slope (degrees)')
Map.addLayer(mountain_areas, mountain_vis, 'Mountain Classification (UNEP)')

legend_dict = {
    'Type 1 (>4500m)': '#d73027',
    'Type 2 (3500-4500m)': '#f46d43', 
    'Type 3 (2500-3500m)': '#fdae61',
    'Type 4 (1500-2500m, slope≥2°)': '#fee08b',
    'Type 5 (1000-1500m, conditions)': '#d9ef8b',
    'Type 6 (300-1000m, conditions)': '#66bd63'
}
Map.add_legend(legend_dict=legend_dict, title="Mountain Types (UNEP)")

# 显示地图
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…